In [8]:
# -*- coding: utf-8 -*-
import random
import torch
import math
device = torch.device("cuda:0") # Uncomment this to run on GPU


class DynamicNet(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate five parameters and assign them as members.
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))
        self.e = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        """
        For the forward pass of the model, we randomly choose either 4, 5
        and reuse the e parameter to compute the contribution of these orders.

        Since each forward pass builds a dynamic computation graph, we can use normal
        Python control-flow operators like loops or conditional statements when
        defining the forward pass of the model.

        Here we also see that it is perfectly safe to reuse the same parameter many
        times when defining a computational graph.
        """
        y = self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3
        for exp in range(4, random.randint(4, 6)):
            y = y + self.e * x ** exp
        return y

    def string(self):
        """
        Just like any class in Python, you can also define custom method on PyTorch modules
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3 + {self.e.item()} x^4 ? + {self.e.item()} x^5 ?'


# Create Tensors to hold input and outputs.
# x = torch.linspace(-math.pi, math.pi, 2000)
x = torch.linspace(-math.pi, math.pi, 2000, device=device)
y = torch.sin(x)

# Construct our model by instantiating the class defined above
# model = DynamicNet()
model = DynamicNet().to(device)

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-8, momentum=0.9)

import time

start = time.time() 
for t in range(30000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 2000 == 1999:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}', time.time() -start)

1999 626.1844482421875
3999 305.780029296875
5999 141.49761962890625
7999 1813.0030517578125
9999 36.104957580566406
11999 22.02686309814453
13999 14.813693046569824
15999 11.627144813537598
17999 10.130712509155273
19999 9.423761367797852
21999 9.110570907592773
23999 8.788155555725098
25999 8.700435638427734
27999 8.608137130737305
29999 8.858768463134766
Result: y = -0.0028418696019798517 + 0.85528564453125 x + 9.526364010525867e-06 x^2 + -0.09343798458576202 x^3 + 5.4861266107764095e-05 x^4 ? + 5.4861266107764095e-05 x^5 ? 18.381731748580933
